# Day 19

## part 1

- existing towel patterns are shown
- and towel designs are listed below
- can use as many instances of a pattern to make designs
- find out how many designs are possible

In [9]:
from dataclasses import dataclass
import logging
import re
import random

from tqdm import tqdm

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 19')
logging.basicConfig(level=logging.INFO)

In [2]:
parser = PC('\n\n', [
    PC(', ', str),
    PC('\n', str)
])
patterns, designs = parse_from_file('day_19.txt', parser)

INFO:advent_of_code_utils.py:2 items loaded from "day_19.txt"


In [3]:
log.info(f'Towels with length 1: {[t for t in patterns if len(t) == 1]}')

INFO:day 19:Towels with length 1: ['b', 'w', 'u', 'r']


ok great that means its just about finding how to deal with the `g`s as everything else is trivial

In [4]:
g_patterns = [t for t in patterns if 'g' in t]
log.info(f'Found {len(g_patterns)} towels with "g"')
test_patterns = g_patterns + [t for t in patterns if len(t) == 1]

INFO:day 19:Found 266 towels with "g"


In [5]:
# lets try this tomorrow
# go through the patterns from biggest to smallest
# for each one try and fit it into the design
# if it fits blank out where it fits and continue
# if it fits more than once try all combinations (except for length 1)
# if the whole design gets blanked out return the solution
# else if all strings are tried, blacklist what was tried
# if all len > 1 strings are blacklisted then this desigin won't work

In [6]:
def test_design(
    design: str, patterns: list[str]
) -> dict[str: list[int]] | None:
    """returns the combination of patterns that creates the design, or None"""
    log.debug(f'{design=}')
    working = ''.join(design)
    solution = {}
    # try patterns > len 1
    log.debug(f'testing patterns len > 1')
    subset = [p for p in patterns if len(p) > 1]
    for pattern in subset:
        # log.debug(f'testing {pattern=}')
        if pattern in working:
            solution.update({pattern: []})
        for match in re.finditer(pattern, working):
            log.debug(f'{match=}')
            temp = ''
            start, end = match.span()
            for index, char in enumerate(working):
                if start <= index < end:
                    temp += char.upper()
                else:
                    temp += char
            solution[pattern].append(start)
            working = temp
            log.debug(f'{working=}')
            log.debug(f'{solution=}')
    # try patterns == len 1
    log.debug(f'testing patterns len == 1')
    subset = [p for p in patterns if len(p) == 1]
    for pattern in subset:
        # log.debug(f'testing {pattern=}')
        if pattern in working:
            solution.update({pattern: []})
        for match in re.finditer(pattern, working):
            log.debug(f'{match=}')
            temp = ''
            start, end = match.span()
            for index, char in enumerate(working):
                if start <= index < end:
                    temp += char.upper()
                else:
                    temp += char
            solution[pattern].append(start)
            working = temp
            log.debug(f'{working=}')
            log.debug(f'{solution=}')
    # test the outcome
    if working == working.upper():
        log.info(f'solution found {design=}')
        return solution
    log.info(f'No solution for {design=}')
    return None

# lets try using all the patterns
log.setLevel(logging.WARNING)
solutions = {}
for design in designs:
    solutions.update({design: test_design(design, test_patterns)})
solvable = len([s for s in solutions.values() if s is not None])
log.warning(f'{solvable} solvable found')

In [7]:
# lets try removing one pattern and resolving
log.setLevel(logging.WARNING)
for design in tqdm([d for d, s in solutions.items() if s is None], desc='testing unsolvables'):
    for pattern in test_patterns:
        log.info(f'removing {pattern=}')
        temp_patterns = [p for p in test_patterns if p != pattern]
        solution = test_design(design, temp_patterns)
        if solution is not None:
            break
    if solution is not None:
        log.info('solution found!')
        solutions[design] = solution
solvable = len([s for s in solutions.values() if s is not None])
log.warning(f'{solvable} solvable found')

testing unsolvables:   0%|          | 0/133 [00:00<?, ?it/s]

testing unsolvables: 100%|██████████| 133/133 [00:34<00:00,  3.84it/s]


In [ ]:
# lets try removing randomising the pattern order
log.setLevel(logging.WARNING)
for design in tqdm([d for d, s in solutions.items() if s is None], desc='testing unsolvables'):
    # we'll run it a bunch of times just to see
    for _ in range(100):
        temp_patterns = [p for p in test_patterns]
        random.shuffle(temp_patterns)
        solution = test_design(design, temp_patterns)
        if solution is not None:
            break
    if solution is not None:
        log.info('solution found!')
        solutions[design] = solution
solvable = len([s for s in solutions.values() if s is not None])
log.warning(f'{solvable} solvable found')

testing unsolvables: 100%|██████████| 58/58 [00:06<00:00,  8.34it/s]


In [14]:
solvable = len([s for s in solutions.values() if s is not None])
markdown(f'the number of solvable designs: {solvable}')

the number of solvable designs: 342